In [2]:
import os
import numpy as np
import pandas as pd
import random

In [195]:
class AssetOverlap():
    def __init__(self):
        self.readFunds()
        self.readSP500()

        
    def readFunds(self):
        fundDir = os.listdir("../input/funds")
        funds = pd.DataFrame(columns=["Ticker", "ID"])
        for name in fundDir:
            fund = pd.read_csv("../input/funds/"+name, index_col=0)
            name = name.split(".csv")[0]
            
            fund = fund.rename(columns={fund.columns[-1]:"Ticker", "ID.WEIGHTS":name})
            fund = fund[["ID","Ticker",name]]
            fund = fund[(fund.Ticker.str[:4]!="#N/A") & (fund[name]>0)] #no puts so neg percent and #N/A Unclassified: Unable to parse request at ...  and # N/A Invalid Security
            fund = fund.dropna(axis=0, how='all')
            
            fund[name] = fund[name]/(fund[name].sum())*100 # some have over 100 percent
            
            fund.Ticker = fund.Ticker.fillna("NULL TICKER"+fund.ID)
            
            fund = fund.groupby(["Ticker","ID"]).agg({name:"sum"})# you have to groupby id and ticker or else it may merge multipletimes
            
            funds = funds.merge(fund, how='outer', on=["Ticker","ID"])
            
            params = {fundName:"sum" for fundName in self.getFundNames(funds)}
            params.update({"ID":"first"})
            funds = funds.groupby("Ticker").agg(params)
            funds = funds.reset_index()

            #print(funds.Ticker.duplicated().sum())
        
        funds = funds.fillna(0)
        print(funds.sum())
        self.funds = funds

    def getFundMatrix(self):
        return self.funds.loc[:,~self.funds.columns.isin(["ID","Ticker"])]
    
    def readSP500(self):
        self.sp500 = pd.read_csv("../input/s&p500.csv")
        self.sp500 = self.sp500.iloc[:,:3]
        self.sp500 = self.sp500.rename(columns={"Symbol":"Ticker", "Weight":"S&P500"})
        
    def getFundNames(self,funds):
        return [x for x in funds.columns if x not in ["ID", "Ticker"]]
    
    def makePortfolio(self,weights):
        #self.weights = self.weights.loc[self.getFundNames(self.funds)]
        columns = ["Ticker","ID"] + list(weights.index)
        self.funds = self.funds[columns]

        Portfolio = self.getFundMatrix().dot(weights)
        print(Portfolio.sum())
        Portfolio = Portfolio.rename({Portfolio.columns[0]:"Portfolio"},axis=1)
        Portfolio[["Ticker","ID"]] = self.funds[["Ticker","ID"]]
        
        Portfolio = pd.merge(Portfolio, self.sp500, how='outer', on="Ticker")
        Portfolio.ID = Portfolio.ID.fillna(Portfolio.Company)
        Portfolio = Portfolio.drop("Company", axis=1)
        return Portfolio
    
    def makeAddStocksPortfolio(self,Portfolio):
        dif = Portfolio["S&P500"]-Portfolio["Portfolio"] # determine which sp500 assets have more than in the portfolio
        self.extra = Portfolio
        self.extra["Dif"] = dif
        self.extra = self.extra[self.extra.Dif>0][["Ticker","ID","Dif"]]
        self.extra = self.extra.rename(columns={"Dif":"Portfolio"})
        AddStocksPortfolio = pd.concat([Portfolio[["Ticker","ID","Portfolio"]],self.extra],axis=0)
        return AddStocksPortfolio
    
    def simpleOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        Portfolio[["Portfolio","S&P500"]] = Portfolio[["Portfolio","S&P500"]].fillna(0)
        overlap = np.minimum(np.array(Portfolio["Portfolio"]),np.array(Portfolio["S&P500"]))
        overlap = overlap.sum()
        total = Portfolio.Portfolio.sum()
        print(total)
        return overlap/total*100
    
    def addStocksOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        AddStocksPortfolio = self.makeAddStocksPortfolio(Portfolio)
        return 100/AddStocksPortfolio.Portfolio.sum()*100
        
    def evaluateAssetOverlap(self,weights):
        #self.weights = pd.DataFrame(np.ones((49,1))*-1000,index=self.getFundNames(self.funds))
        return self.simpleOverlap(weights)
        
        
        
        

In [221]:
fundDir = os.listdir("../input/funds")
funds = pd.DataFrame(columns=["Ticker", "ID"])
for name in fundDir:
    fund = pd.read_csv("../input/funds/"+name, index_col=0)
    name = name.split(".csv")[0]

    fund = fund.rename(columns={fund.columns[-1]:"Ticker", "ID.WEIGHTS":name})
    fund = fund[["ID","Ticker",name]]
    fund = fund[(fund.Ticker.str[:4]!="#N/A") & (fund[name]>0)] #no puts so neg percent and #N/A Unclassified: Unable to parse request at ...  and # N/A Invalid Security
    fund = fund.dropna(axis=0, how='all')

    fund[name] = fund[name]/(fund[name].sum())*100 # some have over 100 percent

    fund.Ticker = fund.Ticker.fillna("NULL TICKER"+fund.ID)

    fund = fund.groupby(["Ticker","ID"]).agg({name:"sum"})# you have to groupby id and ticker or else it may merge multipletimes

    funds = funds.merge(fund, how='outer', on=["Ticker","ID"])

    params = {fundName:"sum" for fundName in getFundNames(funds)}
    params.update({"ID":"first"})
#     funds = funds.groupby("Ticker").agg(params)
#     funds = funds.reset_index()

    #print(funds.Ticker.duplicated().sum())

funds = funds.fillna(0)

k = funds.groupby(["Ticker"])["ID"].count().to_frame()

t = funds.groupby(["Ticker","ID"])["BUI"].count()
t.tail(20)
#t[t.index.get_level_values(0).isin(k[k.ID>1].index)].to_frame()

Ticker  ID               
YNDX    YNDX US Equity       1
YUM     YUM US Equity        1
YUMC    YUMC US Equity       1
YY      YY US Equity         1
Z       AT341057     Corp    1
        BJ456485     Corp    1
ZBH     ZBH US Equity        1
ZBRA    ZBRA US Equity       1
ZEN     BK051856     Corp    1
        ZEN US Equity        1
ZI      ZI US Equity         1
ZLAB    ZLAB US Equity       1
ZNGA    AZ096782     Corp    1
        ZNGA US Equity       1
ZNTL    ZNTL US Equity       1
ZS      BK199428     Corp    1
        ZS US Equity         1
ZTS     ZTS US Equity        1
ZURN    ZURN SW Equity       1
ZYME    ZYME US Equity       1
Name: BUI, dtype: int64

In [269]:
def bestID(ids):
    return ids.iloc[np.argmax([np.all([not char.isdigit() for char in ID]) for ID in ids])]

In [270]:
funds.groupby("Ticker").agg({"ID":bestID})

ID
Ticker                  
000333  000333 CH Equity
000858  000858 CH Equity
002179  002179 CH Equity
002371  002371 CH Equity
002475  002475 CH Equity
...                  ...
ZNTL      ZNTL US Equity
ZS          ZS US Equity
ZTS        ZTS US Equity
ZURN      ZURN SW Equity
ZYME      ZYME US Equity

[1722 rows x 1 columns]

In [283]:
funds

Ticker                  ID       STK       ADX  CHN  MIE  SZC       CII  \
0        42  0758519D US Equity  1.762070  0.000000  0.0  0.0  0.0  0.000000   
1      6723      6723 JP Equity  1.364854  0.000000  0.0  0.0  0.0  0.000000   
2      AAPL      AAPL US Equity  5.840233  6.361546  0.0  0.0  0.0  7.510249   
3       ADI       ADI US Equity  0.926434  0.000000  0.0  0.0  0.0  0.000000   
4      AEIS      AEIS US Equity  1.265053  0.000000  0.0  0.0  0.0  0.000000   
...     ...                 ...       ...       ...  ...  ...  ...       ...   
2333    TCS       TCS IN Equity  0.000000  0.000000  0.0  0.0  0.0  0.000000   
2334   TLGF      TLGF TB Equity  0.000000  0.000000  0.0  0.0  0.0  0.000000   
2335  VIVT3     VIVT3 BZ Equity  0.000000  0.000000  0.0  0.0  0.0  0.000000   
2336   WUBA      WUBA US Equity  0.000000  0.000000  0.0  0.0  0.0  0.000000   
2337   YUMC      YUMC US Equity  0.000000  0.000000  0.0  0.0  0.0  0.000000   

      GRF  BCX  ...  BMEZ       CET  UTF  BOE  IGR  PEO  DNP  RQI       AEF  \
0     0.0  0.0  ...   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.000000   
1     0.0  0.0  ...   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.000000   
2     0.0  0.0  ...   0.0  0.933408  0.0  0.0  0.0  0.0  0.0  0.0  0.000000   
3     0.0  0.0  ...   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.000000   
4     0.0  0.0  ...   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.000000   
...   ...  ...  ...   ...       ...  ...  ...  ...  ...  ...  ...       ...   
2333  0.0  0.0  ...   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  1.234798   
2334  0.0  0.0  ...   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  1.208732   
2335  0.0  0.0  ...   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  1.129441   
2336  0.0  0.0  ...   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  1.057430   
2337  0.0  0.0  ...   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.624688   

      BUI  
0     0.0  
1     0.0  
2     0.0  
3     0.0  
4     0.0  
...   ...  
2333  0.0  
2334  0.0  
2335  0.0  
2336  0.0  
2337  0.0  

[2338 rows x 51 columns]

In [303]:
def hello(x):
    for y in x:
        if "NULL" not in y:
            if np.any(["NULL" in y for y in x]):
                print("WOO")
            return y
    else: 
        return x.iloc[0]
    

In [304]:
k = funds.groupby("ID")["Ticker"].apply(hello)
k

ID
000333 CH Equity     000333
000858 CH Equity     000858
002179 CH Equity     002179
002371 CH Equity     002371
002475 CH Equity     002475
                      ...  
ZS810219     Corp      MTCH
ZS891212     Corp      GDDY
ZTS US Equity           ZTS
ZURN SW Equity         ZURN
ZYME US Equity         ZYME
Name: Ticker, Length: 2338, dtype: object

Ticker_x  \
1053  NULL TICKERAFFERENT MILESTONE INTEREST COMMON ...   
1054  NULL TICKERBRSZ39GT0 PROJECT RADAR PROJECT RAD...   
1055  NULL TICKERBRW1R4Y73 PROJECT BEADS PROJECT BEA...   
1056  NULL TICKERBRW5236Q2 PROJECT K 1.18 PROJECT K ...   
1057  NULL TICKERBRW620EA2 PROJECT Q PROJECT Q SERIES D   
1058  NULL TICKERBRW73BE07 PROJ BRIAN PROJECT BRAIN ...   
1059  NULL TICKERBRW7G6UM8 PROJECT TF CON PROJECT TF...   
1060  NULL TICKERMERRILL LYNCH LIQUIDITY SER LLCMONE...   
1061                                     NULL TICKEROAK   
1540                      NULL TICKER1050346D AU Equity   
1541                      NULL TICKER5158754Z SS Equity   
1542  NULL TICKERC3 AI INC SERIES H  (ACQUIRED 08/14...   
1543  NULL TICKERCNEX LABS INC SERIES E  (ACQUIRED 0...   
1544  NULL TICKERCREDO TECHNOLOGY GROUP HOLDING LTD ...   
1545  NULL TICKERDATABRICKS INC SERIES F  (ACQUIRED ...   
1546  NULL TICKERGITLAB INC SERIES E  (ACQUIRED 09/1...   
1547  NULL TICKERINNOVIUM INC SERIES E  (ACQUIRED 08...   
1548  NULL TICKERINNOVIUM INC SERIES F  (ACQUIRED 06...   
1549  NULL TICKERSAMBANOVA SYSTEMS INC SERIES C  (AC...   
1550  NULL TICKERSL LIQUIDITY SERIES LLC MONEY MARKE...   
1551  NULL TICKERTRAX LTD  (ACQUIRED 09/12/19 COST $...   
1552  NULL TICKERTRAX LTD  (ACQUIRED 09/12/2019 COST...   
1553  NULL TICKERUNQORK INC SERIES B  (ACQUIRED 09/1...   
2063  NULL TICKERBRW2GHZB4 PROJECT DIRECT PROJECT EL...   
2064         NULL TICKERBRW4JPE89 PROJECT BD PROJECT DB   
2065  NULL TICKERBRW4TDHW9 PROJ TARGET PROJECT TARGE...   
2066  NULL TICKERBRW8TT9L8 PROJECT NEWER PROJECT NEW...   
2067                          NULL TICKERMNTA US Equity   
2068  NULL TICKEROCUMENSION THERAPEUTICS COMMON STOC...   
2069             NULL TICKERPROJECT MM C1 EQUITY/Equity   
2070                           NULL TICKERPROJECT MM C2   
2071  NULL TICKERTALARIS  THERAPEUTICS PROJECT TRANS...   

                                                     ID  STK  ADX  CHN  MIE  \
1053           AFFERENT MILESTONE INTEREST COMMON STOCK  0.0  0.0  0.0  0.0   
1054     BRSZ39GT0 PROJECT RADAR PROJECT RADAR COMMON B  0.0  0.0  0.0  0.0   
1055     BRW1R4Y73 PROJECT BEADS PROJECT BEADS SERIES B  0.0  0.0  0.0  0.0   
1056        BRW5236Q2 PROJECT K 1.18 PROJECT K SERIES B  0.0  0.0  0.0  0.0   
1057             BRW620EA2 PROJECT Q PROJECT Q SERIES D  0.0  0.0  0.0  0.0   
1058        BRW73BE07 PROJ BRIAN PROJECT BRAIN SERIES B  0.0  0.0  0.0  0.0   
1059    BRW7G6UM8 PROJECT TF CON PROJECT TF CONVERTIBLE  0.0  0.0  0.0  0.0   
1060  MERRILL LYNCH LIQUIDITY SER LLCMONEY MARKET SER 1  0.0  0.0  0.0  0.0   
1061                                                OAK  0.0  0.0  0.0  0.0   
1540                                 1050346D AU Equity  0.0  0.0  0.0  0.0   
1541                                 5158754Z SS Equity  0.0  0.0  0.0  0.0   
1542  C3 AI INC SERIES H  (ACQUIRED 08/14/19 COST $2...  0.0  0.0  0.0  0.0   
1543  CNEX LABS INC SERIES E  (ACQUIRED 02/06/20  $7...  0.0  0.0  0.0  0.0   
1544  CREDO TECHNOLOGY GROUP HOLDING LTD SERIES D  (...  0.0  0.0  0.0  0.0   
1545  DATABRICKS INC SERIES F  (ACQUIRED 10/22/19 CO...  0.0  0.0  0.0  0.0   
1546  GITLAB INC SERIES E  (ACQUIRED 09/10/19  $1167...  0.0  0.0  0.0  0.0   
1547  INNOVIUM INC SERIES E  (ACQUIRED 08/21/19  $89...  0.0  0.0  0.0  0.0   
1548  INNOVIUM INC SERIES F  (ACQUIRED 06/10/20 COST...  0.0  0.0  0.0  0.0   
1549  SAMBANOVA SYSTEMS INC SERIES C  (ACQUIRED 02/2...  0.0  0.0  0.0  0.0   
1550  SL LIQUIDITY SERIES LLC MONEY MARKET SERIES (G...  0.0  0.0  0.0  0.0   
1551     TRAX LTD  (ACQUIRED 09/12/19 COST $0) WARRANTS  0.0  0.0  0.0  0.0   
1552  TRAX LTD  (ACQUIRED 09/12/2019 COST $10999988)...  0.0  0.0  0.0  0.0   
1553  UNQORK INC SERIES B  (ACQUIRED 09/19/19  $6801...  0.0  0.0  0.0  0.0   
2063  BRW2GHZB4 PROJECT DIRECT PROJECT ELIMINATE SERIES  0.0  0.0  0.0  0.0   
2064                    BRW4JPE89 PROJECT BD PROJECT DB  0.0  0.0  0.0  0.0   
2065        BRW4TDHW9 PROJ TARGET PROJECT TAR

In [259]:
def hey(t):
    return t.iloc[np.argmax([np.all([not x.isdigit() for x in y]) for y in t])]

In [265]:
#tickers where there is more than one id
k = funds.groupby(["Ticker"])["ID"].count().to_frame()
t = funds.groupby(["Ticker","ID"])["BUI"].count()
t[t.index.get_level_values("Ticker").isin(k[k.ID>1].index)].reset_index().groupby("Ticker")["ID"].apply(hey)

Ticker
42        0758519D US Equity
AAL            AAL LN Equity
ACACN      AH028916     Corp
ACAFP      AX329820     Corp
ACGCAP     AP547793     Corp
                 ...        
WRB         EP050832     Pfd
Z          AT341057     Corp
ZEN        BK051856     Corp
ZNGA       AZ096782     Corp
ZS         BK199428     Corp
Name: ID, Length: 297, dtype: object

In [240]:
t.iloc[-20:]

Ticker  ID               
YNDX    YNDX US Equity       1
YUM     YUM US Equity        1
YUMC    YUMC US Equity       1
YY      YY US Equity         1
Z       AT341057     Corp    1
        BJ456485     Corp    1
ZBH     ZBH US Equity        1
ZBRA    ZBRA US Equity       1
ZEN     BK051856     Corp    1
        ZEN US Equity        1
ZI      ZI US Equity         1
ZLAB    ZLAB US Equity       1
ZNGA    AZ096782     Corp    1
        ZNGA US Equity       1
ZNTL    ZNTL US Equity       1
ZS      BK199428     Corp    1
        ZS US Equity         1
ZTS     ZTS US Equity        1
ZURN    ZURN SW Equity       1
ZYME    ZYME US Equity       1
Name: BUI, dtype: int64

In [219]:
k.tail(20)

ID
Ticker    
XPER     1
XRIFX    1
XRO      1
XTTPX    1
YNDX     1
YUM      1
YUMC     1
YY       1
Z        2
ZBH      1
ZBRA     1
ZEN      2
ZI       1
ZLAB     1
ZNGA     2
ZNTL     1
ZS       2
ZTS      1
ZURN     1
ZYME     1

In [198]:
AssetOverlap()

Ticker    0003330008580021790023710024750027470059350064...
STK                                                   100.0
ADX                                                   100.0
CHN                                                   100.0
MIE                                                   100.0
SZC                                                   100.0
CII                                                   100.0
GRF                                                   100.0
BCX                                                   100.0
CRF                                                   100.0
CEN                                                   100.0
ASA                                                   100.0
AGD                                                   100.0
AIO                                                   100.0
RFI                                                   100.0
BME                                                   100.0
CPZ                                     

In [153]:
 def getFundNames(funds):
    return [x for x in funds.columns if x not in ["ID", "Ticker"]]

In [185]:
fundDir = os.listdir("../input/funds")
funds = pd.DataFrame(columns=["Ticker", "ID"])

for name in fundDir[0:40]:
    fund = pd.read_csv("../input/funds/"+name, index_col=0)
    name = name.split(".csv")[0]

    fund = fund.rename(columns={fund.columns[-1]:"Ticker", "ID.WEIGHTS":name})
    fund = fund[["ID","Ticker",name]]
    fund = fund[(fund.Ticker.str[:4]!="#N/A") & (fund[name]>0)] #no shorts so neg percent and #N/A Unclassified: Unable to parse request at ...  and # N/A Invalid Security
    fund = fund.dropna(axis=0, how='all')

    fund[name] = fund[name]/(fund[name].sum())*100
    print(name, fund[name].sum())
    #fund.loc[fund["Ticker"].isnull(),"Ticker"] = "NULL TICKER"
    fund.Ticker = fund.Ticker.fillna("NULL TICKER")
    if name == "BMEZ":
        tickers = fund.groupby(["Ticker"])["Ticker"].count().index
        print(fund.Ticker.isin(tickers))
        print(fund)
    fund = fund.groupby(["Ticker"]).agg({name:"sum"})
    if name == "BMEZ":
        print(fund)
    print(name, fund[name].sum())
#     funds = funds.merge(fund, how='outer', on=["Ticker","ID"])

#     params = {fundName:"sum" for fundName in getFundNames(funds)}
#     params.update({"ID":"first"})
#     funds = funds.groupby("Ticker").agg(params)
#     funds = funds.reset_index()

    #print(funds.Ticker.duplicated().sum())

funds = funds.fillna(0)
print(funds.sum())

STK 100.00000000000001
STK 100.00000000000001
ADX 100.0
ADX 100.0
CHN 100.0
CHN 100.0
MIE 100.0
MIE 100.0
SZC 100.0
SZC 100.0
CII 100.0
CII 100.0
GRF 100.0
GRF 99.99999999999999
BCX 99.99999999999999
BCX 100.0
CRF 100.0
CRF 100.0
CEN 100.0
CEN 99.99999999999999
ASA 100.0
ASA 100.0
AGD 100.0
AGD 100.00000000000001
AIO 100.0
AIO 99.99999999999999
RFI 100.0
RFI 99.99999999999999
BME 100.00000000000001
BME 100.0
CPZ 100.00000000000001
CPZ 100.00000000000003
CEM 100.0
CEM 100.0
BSTZ 100.0
BSTZ 99.99999999999999
NFJ 100.0
NFJ 100.0
RNP 100.00000000000001
RNP 100.0
EXD 100.0
EXD 99.99999999999999
EOI 100.0
EOI 100.0
CTR 100.0
CTR 99.99999999999999
GLQ 100.0
GLQ 99.99999999999999
BGR 100.00000000000003
BGR 100.0
JEQ 100.0
JEQ 99.99999999999999
DSE 100.00000000000001
DSE 100.00000000000001
BDJ 100.00000000000001
BDJ 100.0
SRV 100.0
SRV 99.99999999999999
CLM 100.00000000000001
CLM 100.00000000000001
EMO 100.00000000000001
EMO 100.00000000000001
DPG 99.99999999999999
DPG 99.99999999999996
AWP 100

In [156]:
class Discount:
    def __init__(self):
        self.readDiscount()
    def readDiscount(self):
        self.discount = pd.read_csv("../input/Discounts.csv")
        self.discount[["Discount", "52W Discount","Effective","Distribution"]] = self.discount.apply(lambda x: [x["Discount"].strip("%"),x["52W Discount"].strip("%"), x["Effective"].strip("%"), x["Distribution"].strip("%")], axis=1, result_type='expand')
        self.discount = self.discount.replace("--",0)
        self.discount.iloc[:,1:] = self.discount.iloc[:,1:].astype("float")
        
        longterm = self.discount["52W Discount"]-self.discount["Discount"] #max: a pos 52 week is pos, a neg discount is more pos
        longterm = longterm/np.max(longterm) *100# between -inf and 1
        self.discount["longterm"] = longterm
        current = -self.discount["Discount"] # max: a neg discount is pos
        current = current/np.max(-self.discount["Discount"]) * 100 #between -inf and 1
        self.discount["value"] = np.sum([longterm,2*current],axis=0)/3
        self.discount = self.discount.set_index("Ticker")
    
    def getWeightedDiscount(self,weights):
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.Discount).reshape(len(self.discount),1)
        return np.matmul(weights,values)[0][0]
    
    def evaluateDiscount(self,weights):
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.value).reshape(len(self.discount),1)
        
        return np.dot(weights,values)[0][0]
        
          

In [109]:
Discount().discount

Effective  Distribution  Discount  52W Discount    longterm      value
Ticker                                                                        
ADX          0.00         20.55    -13.27        -13.74   -4.304029  36.468780
AEF          7.90          2.64    -12.89        -14.09  -10.989011  33.155048
AGD          6.94          7.51    -12.70        -13.11   -3.754579  35.023824
AIO          2.95          5.05     -6.89        -10.25  -30.769231   9.423681
AOD          0.09          7.84    -13.81        -13.28    4.853480  41.063699
ASA          0.00          0.09    -15.96        -15.45    4.670330  47.143752
AWP          7.68          9.28    -12.22        -13.72  -13.736264  30.325558
BCX          1.20          6.63    -14.22        -15.29   -9.798535  37.350788
BDJ          0.27          7.19     -9.53         -9.34    1.739927  27.800770
BGR          0.22          7.88    -13.84         -9.02   44.139194  54.244627
BGY          0.36          6.99    -10.08        -11.68  -14.652015  23.907769
BIF          0.36          3.68    -17.19        -16.86    3.021978  50.107583
BME          0.00          4.93      7.65          1.73  -54.212454 -39.921718
BMEZ         0.06          4.07     -3.06          0.00   28.021978  18.081019
BOE          0.00          7.09    -12.33        -12.30    0.274725  35.310084
BSTZ         0.26          3.83     -6.47         -7.83  -12.454212  14.329030
BUI          0.13          6.01      3.42          1.45  -18.040293 -15.782069
CEM         29.27         10.33    -17.60        -17.72   -1.098901  49.905051
CEN         60.97         26.87    -23.34        -13.78   87.545788  95.848596
CET          0.32          9.43    -18.61        -17.49   10.256410  56.575044
CHN          0.10          0.44     -9.55        -13.63  -37.362637  14.823708
CII          0.10          5.99     -7.00         -8.30  -11.904762  16.026033
CLM          0.00         19.15     18.47          8.21  -93.956044 -84.075037
CPZ         30.58          8.17    -14.01        -12.27   15.934066  45.328493
CRF          9.50         18.94     20.55          9.89  -97.619048 -91.237198
CTR         19.90         10.09    -20.54        -19.16   12.637363  62.881406
DNP         26.66          7.52     14.89         19.66   43.681319 -27.970266
DPG         29.57         12.15    -12.46        -10.69   16.208791  40.992762
DSE         14.84        113.21    -19.21         -8.29  100.000000  88.203370
EMO         27.11          8.92    -22.75        -20.25   22.893773  72.612692
EOI          0.00          6.67     -3.52         -4.23   -6.501832   7.886993
EOS          0.00          5.50      2.57         -0.67  -29.670330 -17.230870
ETJ          0.00          8.69      4.37         -0.48  -44.413919 -27.286788
EXD          0.00          8.49     -4.48         -5.74  -11.538462   8.950190
GLQ         37.40          9.68    -14.17        -13.15    9.340659  43.587703
GRF          0.00          7.54    -22.63        -18.83   34.798535  76.238186
IAF          0.00          9.77    -12.21        -11.10   10.164835  38.264028
IGR         19.62          8.93    -15.15        -15.43   -2.564103  42.418650
JEQ          0.00          2.72    -14.25        -16.11  -17.032967  35.025001
MIE         18.02          7.44    -18.24        -12.00   57.142857  71.147019
NFJ          0.00          6.77    -12.68        -12.63    0.457875  36.370849
PEO          0.70         14.84    -17.04        -13.88   28.937729  58.317718
RFI          1.52          7.23      2.47         -0.37  -26.007326 -15.724236
RNP         23.99          6.80     -6.09         -8.75  -24.358974   9.275372
RQI         22.75          7.90     -5.81         -8.78  -27.197802   7.529324
SRV          3.13          6.37    -22.75        -20.32   22.252747  72.399016
STK          0.00          6.70      0.80          2.16   12.454212   1.866343
SZC          3.02          6.89    -23.32        -21.58   15.934066  71.920895
UTF         27.11          7.21      5.61         -3.19  -

In [88]:
class runModel(AssetOverlap, Discount):
    def __init__(self):
        self.readFunds()
        self.readSP500()
        self.readDiscount()
        
        N = len(self.getFundNames(self.funds))
        weights = np.random.uniform(0,1.0,(N,1))
        weights = np.divide(weights,np.sum(weights))

        self.weights = pd.DataFrame(weights,columns=["weights"],index=self.getFundNames(self.funds))
        
        #self.findInitialBest()
    
    def Evaluate(self,weights):
        asset = 1*(self.evaluateAssetOverlap(weights))
        discount = 0*-self.evaluateDiscount(weights)
        return asset+discount
    
    def changeWeights(self,fund1,fund2,weights,change):
        change = min(weights.loc[fund2][0],change)
        weights.loc[fund1,"weights"] += change
        weights.loc[fund2,"weights"] -= change
        return weights
    
    def EvaluateChange(self,fund1,fund2,weights,change):
        weights = self.changeWeights(fund1,fund2,weights,change)
        return self.Evaluate(weights)
    
    def findIdealChange(self,fund1,fund2,weights,change):
        control = self.Evaluate(weights)
        while self.EvaluateChange(fund1,fund2,weights,change)>control and self.weights.loc[fund2]!=0:
            weights = self.changeWeights()
        return control, weights
    
    def makeAdjustment(self,fund1,fund2,weights,change):
        control = self.Evaluate(weights)
        up = self.EvaluateChange(fund1,fund2,weights,change)
        down = self.EvaluateChange(fund2,fund1,weights,change)
        
        if down>control and down>up:
            fund1, fund2 = fund2, fund1
        elif control>up:
            return control,weights
        
        weights = self.changeWeights(fund1,fund2,weights,change)
        value, weights = self.findIdealChange(fund1,fund2,weights,change)
        
        return value,weights
    
    def randomFunds(self):
        fund2 = random.choice(self.weights.index)
        fund1 = random.choice(self.weights.index)
        return fund1, fund2
    
    def trial(self):
        fund1, fund2 = self.randomFunds()
        change = .1
        print(obj.weights.sum())
        print(self.Evaluate(self.weights))
        value, weights = self.makeAdjustment(fund1, fund2, self.weights, .1)
        print(obj.weights.sum())
        print(value)
        print(weights.loc[fund1], weights.loc[fund2])
    def findInitialBest(self):
        
        options = [self.addValueToFund(1,fund) for fund in self.weights.index]
        print(options)  
        
        

In [89]:
obj = runModel()
obj.trial()

weights    1.0
dtype: float64
106.41307826303483
29.33530743860453
106.41307826303483
106.70900506382624
105.91123259192975
105.91123259192975
105.91123259192975
weights    1.0
dtype: float64
29.525509909355467
weights    0.0
Name: DSE, dtype: float64 weights    0.055545
Name: MIE, dtype: float64


In [110]:
obj.weights.sum()

weights    1.0
dtype: float64

In [480]:
null = funds[funds["Ticker"].str[:4]=="NULL"]
notnull = funds[funds["Ticker"].str[:4]!="NULL"].groupby("ID")["Ticker"].first()
null.merge(notnull, how="left", left_on="ID", right_index=True)
#no overlap between null and having ticker

weights
STK   0.029072
ADX   0.223149
CHN   0.000000
MIE   0.000000
SZC   0.000000
CII   0.049814
GRF   0.017187
BCX   0.000000
CRF   0.153040
CEN   0.000000
ASA   0.000000
AGD   0.000000
AIO   0.000000
RFI   0.000000
BME   0.044555
CPZ   0.000000
CEM   0.000000
BSTZ  0.000000
NFJ   0.000000
RNP   0.000000
EXD   0.000000
EOI   0.071186
CTR   0.000000
GLQ   0.000000
BGR   0.000000
JEQ   0.000000
DSE   0.000000
BDJ   0.050000
SRV   0.000000
CLM   0.109580
EMO   0.000000
DPG   0.000000
AWP   0.000000
EOS   0.101574
BIF   0.014613
ETJ   0.000000
IAF   0.000000
AOD   0.000000
BGY   0.000000
BMEZ  0.000000
CET   0.075365
UTF   0.000000
BOE   0.013463
IGR   0.000000
PEO   0.037403
DNP   0.000000
RQI   0.010000
AEF   0.000000
BUI   0.000000

In [403]:
Discount().discount

Effective  Distribution  Discount  52W Discount      value
Ticker                                                            
ADX          0.00         20.55    -13.27        -13.74 -26.275577
AEF          7.90          2.64    -12.89        -14.09 -22.119033
AGD          6.94          7.51    -12.70        -13.11 -25.329223
AIO          2.95          5.05     -6.89        -10.25   0.624547
AOD          0.09          7.84    -13.81        -13.28 -32.011144
ASA          0.00          0.09    -15.96        -15.45 -36.525396
AWP          7.68          9.28    -12.22        -13.72 -19.310103
BCX          1.20          6.63    -14.22        -15.29 -25.563458
BDJ          0.27          7.19     -9.53         -9.34 -21.285559
BGR          0.22          7.88    -13.84         -9.02 -51.718269
BGY          0.36          6.99    -10.08        -11.68 -14.267823
BIF          0.36          3.68    -17.19        -16.86 -38.336182
BME          0.00          4.93      7.65          1.73  43.494402
BMEZ         0.06          4.07     -3.06          0.00 -20.566259
BOE          0.00          7.09    -12.33        -12.30 -26.551244
BSTZ         0.26          3.83     -6.47         -7.83  -7.633219
BUI          0.13          6.01      3.42          1.45  16.346625
CEM         29.27         10.33    -17.60        -17.72 -37.154063
CEN         60.97         26.87    -23.34        -13.78 -93.772894
CET          0.32          9.43    -18.61        -17.49 -44.995386
CHN          0.10          0.44     -9.55        -13.63  -1.777122
CII          0.10          5.99     -7.00         -8.30  -9.043335
CLM          0.00         19.15     18.47          8.21  86.545288
CPZ         30.58          8.17    -14.01        -12.27 -37.979886
CRF          9.50         18.94     20.55          9.89  92.832660
CTR         19.90         10.09    -20.54        -19.16 -50.320395
DNP         26.66          7.52     14.89         19.66  10.057370
DPG         29.57         12.15    -12.46        -10.69 -34.796769
DSE         14.84        113.21    -19.21         -8.29 -91.152528
EMO         27.11          8.92    -22.75        -20.25 -60.182962
EOI          0.00          6.67     -3.52         -4.23  -4.289787
EOS          0.00          5.50      2.57         -0.67  20.340735
ETJ          0.00          8.69      4.37         -0.48  31.568571
EXD          0.00          8.49     -4.48         -5.74  -3.828027
GLQ         37.40          9.68    -14.17        -13.15 -35.025942
GRF          0.00          7.54    -22.63        -18.83 -65.878273
IAF          0.00          9.77    -12.21        -11.10 -31.239230
IGR         19.62          8.93    -15.15        -15.43 -31.172962
JEQ          0.00          2.72    -14.25        -16.11 -22.010509
MIE         18.02          7.44    -18.24        -12.00 -67.645979
NFJ          0.00          6.77    -12.68        -12.63 -27.392605
PEO          0.70         14.84    -17.04        -13.88 -50.972721
RFI          1.52          7.23      2.47         -0.37  18.295008
RNP         23.99          6.80     -6.09         -8.75  -0.866785
RQI         22.75          7.90     -5.81         -8.78   1.152457
SRV          3.13          6.37    -22.75        -20.32 -59.862449
STK          0.00          6.70      0.80          2.16  -4.513310
SZC          3.02          6.89    -23.32        -21.58 -57.924188
UTF         27.11          7.21      5.61         -3.19  52.311035

In [199]:
FundAssets().addStocksOverlap()

61.39535179302859

In [198]:
FundAssets().simpleOverlap()

33.12524075359048

In [89]:
a = FundAssets().Portfolio.groupby("Company").agg({"S&P500":'count'})

STK
0
ADX
1
CHN
1
MIE
3
SZC
14


In [115]:
dict = {x:"sum" for x in range(2)}
dict.update({"h":'h'})

In [2]:
dict(zip([1,2],["sum","sum"]))

{1: 'sum', 2: 'sum'}

In [1]:
class runModel(AssetOverlap, Discount):
    def __init__(self):
        self.readFunds()
        self.readSP500()
        self.readDiscount()
        
        N = len(self.getFundNames(self.funds))
        weights = np.random.uniform(0,1.0,(N,1))
        weights = np.divide(weights,np.sum(weights))

        self.weights = pd.DataFrame(weights,columns=["weights"],index=self.getFundNames(self.funds))
        
        self.n = .1
        self.run_model()
        
    def Evaluate(self,weights):
        asset = .7*(self.evaluateAssetOverlap(weights))
        discount = .3*-self.evaluateDiscount(weights)
        return asset+discount
        
    def randomFunds(self):
        fund2 = random.choice(self.weights[self.weights["weights"]!=0].index)
        fund1 = random.choice(self.weights.index)
        return fund1, fund2
    
    def run_model(self):
        fund1, fund2 = self.randomFunds()

        for x in range(100):
            newWeights = self.weights.copy()
            
            change = .01
            if newWeights.loc[fund2][0] < change:
                change = newWeights.loc[fund2][0]
            if change == 0 or fund1==fund2: 
                fund1, fund2 = self.randomFunds()
                continue
                
            newWeights.loc[fund1] = [newWeights.loc[fund1]+change]
            newWeights.loc[fund2] = [newWeights.loc[fund2]-change]

            if self.Evaluate(newWeights) > self.Evaluate(self.weights):
                self.weights=newWeights.copy()
            else:
                fund1, fund2 = self.randomFunds()
        
        print(self.evaluateAssetOverlap(self.weights))
        print(self.addStocksOverlap(self.weights))
        print(self.getWeightedDiscount(self.weights))


    

{'a': 1, 'b': 2, 'c': 3}


In [ ]:
# class runModel(AssetOverlap, Discount):
#     def __init__(self):
#         self.readFunds()
#         self.readSP500()
#         self.readDiscount()
        
#         N = len(self.getFundNames(self.funds))
#         weights = np.random.uniform(0,1.0,(N,1))
#         weights = np.divide(weights,np.sum(weights))

#         self.weights = pd.DataFrame(weights,index=self.getFundNames(self.funds))
        
#         self.n = .1
#         self.run_gradient_descent()
    
#     def run_gradient_descent(self):
#         for x in range(100):
#             gradient = -self.n *self.gradient()
#             self.weights = self.weights + gradient 
#             print("k",self.evaluateAssetOverlap())
#             print(self.evaluateDiscount())
            
#     def gradient(self):
#         asset = .5*(100-self.evaluateAssetOverlap())
#         discount = .5*self.evaluateDiscount()
#         print(asset,discount)
#         return asset+discount